In [88]:
from sly import Lexer

class MyLexer(Lexer):
    literals = { '+','-','*','/', "=", ";", ":", ",","(", ")", "'", "{", "}", "[", "]"}
    
    tokens = [
        # Macierzowe operatory binarne
        DOTADD, DOTSUB, DOTMUL, DOTDIV,  
        # Operatory przypisania
        ADDASSIGN, SUBASSIGN, MULASSIGN, DIVASSIGN,
        # Operatory relacyjne
        LT, GT, LE, GE, NE, EQ,  
        # Słowa kluczowe
        IF, ELSE, FOR, WHILE,
        BREAK, CONTINUE, RETURN,
        EYE, ZEROS, ONES,
        PRINT,
        ID, INTNUNM, FLOAT, STRING  # Identyfikatory, liczby, stringi
    ]


    ignore = ' \t'
    # \n jest również ignorowane, ale uwzględnione osobno, by liczyć linie

    @_(r'#.*')
    def ignore_comment(self, t):
        pass

    ID = r'[a-zA-Z_][a-zA-Z0-9_]*'

    DOTADD = r'\.\+'
    DOTSUB = r'\.-'
    DOTMUL = r'\.\*'
    DOTDIV = r'\./'
    ADDASSIGN = r'\+='
    SUBASSIGN = r'-='
    MULASSIGN = r'\*='
    DIVASSIGN = r'/='
    LT = r'<'
    GT = r'>'
    LE = r'<='
    GE = r'>='
    NE = r'!='
    EQ = r'=='

    ID['if'] = IF
    ID['else'] = ELSE
    ID['for'] = FOR
    ID['while'] = WHILE
    ID['break'] = BREAK
    ID['continue'] = CONTINUE
    ID['return'] = RETURN
    ID['eye'] = EYE
    ID['zeros'] = ZEROS
    ID['ones'] = ONES
    ID['print'] = PRINT

    # naukowy float !
    # @_(r'\d+\.\d*|\.\d+')
    @_(r'((\d+\.\d*)|(\.\d+))([eE][-+]?\d+)?')
    def FLOAT(self, t):
        t.value = float(t.value)
        return t

    @_(r'\d+')
    def INTNUM(self, t):
        t.value = int(t.value)
        return t

    @_(r'"[^"]*"')
    def STRING(self, t):
        t.value = t.value[1:-1] 
        return t
    
    @_(r'\n+')
    def newline(self, t):
        self.lineno += t.value.count('\n')


    def error(self, t):
        print(f'Incorrect sign: {t.value[0]} in line: {self.lineno}')
        self.index += 1

if __name__ == '__main__':
    lexer = MyLexer()

    examples = [
    """A = zeros(5); # create 5x5 matrix filled with zeros
    B = ones(7);  # create 7x7 matrix filled with ones
    I = eye(10);  # create 10x10 matrix filled with ones on diagonal and zeros elsewhere
    D1 = A.+B' ;  # add element-wise A with transpose of B
    D2 -= A.-B' ; # substract element-wise A with transpose of B
    D3 *= A.*B' ; # multiply element-wise A with transpose of B
    D4 /= A./B' ; # divide element-wise A with transpose of B
    """,

    """ res1 = 60.500;
    res2 = 60.;
    res3 = .500;
    res4 = 60.52E2;
    str = "Hello world";

    if (m==n) { 
        if (m >= n) 
            print res;
    }
    """,
    
    """ E1 = [ [ 1, 2, 3],
       [ 4, 5, 6],
       [ 7, 8, 9] ];    
    """,

    # own examples
    """ 4.0>$
    """,

    """ printer = 5.0
        print 1.0e-10
        print 5.
        print .53
        if (m == 1)
        K = 1
    """
    ]

    # for i, ex in enumerate(examples):
    i = 4
    ex = examples[i]
    print(f"Przykład {i+1}:\n")
    for tok in lexer.tokenize(ex):
        print(f"({tok.lineno}): {tok.type} ({tok.value})")
    if i+1!= len(examples): print("_____________________________")

Przykład 5:

(1): ID (printer)
(1): = (=)
(1): FLOAT (5.0)
(2): PRINT (print)
(2): FLOAT (1e-10)
(3): PRINT (print)
(3): FLOAT (5.0)
(4): PRINT (print)
(4): FLOAT (0.53)
(5): IF (if)
(5): ( (()
(5): ID (m)
(5): EQ (==)
(5): INTNUM (1)
(5): ) ())
(6): ID (K)
(6): = (=)
(6): INTNUM (1)
